## Import

In [2]:
# script pion.py hjyf
from tkinter import *
import random
from functools import partial

import numpy as np

# visualization

In [3]:

def create_Visualization(titre, env, zoom):
    Mafenetre = Tk() 
    Mafenetre.title(titre)

    # def des couleurs
    myred="#D20B18"
    mygreen="#25A531"
    myblue="#0B79F7"
    mygrey="#E8E8EB"
    myyellow="#F9FB70"
    myblack="#2D2B2B"
    mywalls="#5E5E64"
    mywhite="#FFFFFF"
    color=[mywhite,mygreen,myblue,myred,myblack,mygrey]
    # Creation d'un widget Canvas (pour la grille)
    Largeur = zoom*20*env.nbcolonnes+40
    Hauteur = zoom*20*env.nblignes+40
    # ecriture du quadrillage et coloration
    Canevas = Canvas(Mafenetre, width = Largeur, height =Hauteur, bg =mywhite)
    # cout et affichage
    
    for i in range(env.nblignes+1):
        ni=zoom*20*i+20
        Canevas.create_line(20, ni, Largeur-20,ni)
    for j in range(env.nbcolonnes+1):
        nj=zoom*20*j+20
        Canevas.create_line(nj, 20, nj, Hauteur-20)
    colordraw(env, color, Canevas,zoom)

    
  

    # Creation d'un widget Button (bouton Quitter)
    Button(Mafenetre, text ='Restart', command = create_Visualization).pack(side=LEFT,padx=5,pady=5)
    Button(Mafenetre, text ='Quit', command = Mafenetre.destroy).pack(side=LEFT,padx=5,pady=5)

    w = Label(Mafenetre, text='Cost = '+str(env.globalcost),fg=myblack,font = "Verdana 14 bold")
    w.pack() 

    Pion = Canevas.create_oval(env.PosX-10,env.PosY-10,env.PosX+10,env.PosY+10,width=2,outline='black',fill=myyellow)
    Canevas.coords(Pion,env.PosX -9*zoom, env.PosY -9*zoom, env.PosX +9*zoom, env.PosY +9*zoom)
    w.config(text='Cost = '+ str(env.globalcost))
    process_Clavier = partial(Clavier, env=env, Canevas=Canevas, zoom=zoom, Pion=Pion,w=w)
    Canevas.focus_set()
    Canevas.bind('<Key>',process_Clavier)
    Canevas.pack(padx =5, pady =5)

    env.PosX = 20+10*zoom
    env.PosY = 20+10*zoom
    return Mafenetre
# specification des proportion de murs, case blanches et pts de couleur
def colordraw(env, color, Canevas, zoom):

    
    for i in range(env.nblignes):
        for j in range(env.nbcolonnes):          
            y =zoom*20*i+20
            x =zoom*20*j+20
            if env.g[i,j]>0:
                # print(i,j)
                # print(env.g)
                # print(color[env.g[i,j]])            
                Canevas.create_oval(x+zoom*(10-3),y+zoom*(10-3),x+zoom*(10+3),y+zoom*(10+3),width=1,outline=color[env.g[i,j]],fill=color[env.g[i,j]])
            else:
                if env.g[i,j]<0:
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[4])
                    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[4])
    #ajoute le but
    
    
    y =zoom*20*env.iBut+20
    x =zoom*20*env.jBut+20
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[1])
    Canevas.create_rectangle(x, y, x+zoom*20, y+zoom*20, fill=color[1])

  
def Clavier(event, env, Canevas, zoom, Pion,w):
    
    touche = event.keysym
    cj=int((env.PosX-30)/(20*zoom))
    li=int((env.PosY-30)/(20*zoom))
    changed=0
    cost = np.zeros(6,dtype=np.int)
    for k in range(6):
        cost[k]=0
    # deplacement aleatoire en appuyant sur space
    if touche == 'space':
        t=np.random.randint(6)
        lettre = ['f','g','h','j','y','u',]
        touche=lettre[t]
    # deplacement (-2,1)
    if touche == 'y' and li>1 and cj < env.nbcolonnes-1 and env.g[li-2,cj+1]>-1:
        env.PosY -= zoom*20*2
        env.PosX += zoom*20 
        cost[env.g[li-2,cj+1]]+=1 
        changed=1
    # deplacement (-2,-1)
    if touche == 't' and li>1 and cj > 0 and env.g[li-2,cj-1]>-1:
        env.PosY -= zoom*20*2       
        env.PosX -= zoom*20                 
        cost[env.g[li-2,cj-1]]+=1
        changed=1
   # deplacement (-1,2)
    if touche == 'u' and li>0 and cj < env.nbcolonnes-2 and env.g[li-1,cj+2]>-1:
        env.PosY -= zoom*20        
        env.PosX += zoom*20*2       
        cost[env.g[li-1,cj+2]]+=1
        changed=1
    # deplacement (-1,-2)
    if touche == 'r' and li>0 and cj >1 and env.g[li-1,cj-2]>-1:
        env.PosY -= zoom*20
        env.PosX -= zoom*20*2           
        cost[env.g[li-1,cj-2]]+=1
        changed=1
     # deplacement (2,1)  
    if touche == 'h' and li<env.nblignes-2 and cj < env.nbcolonnes-1 and env.g[li+2,cj+1]>-1:
        env.PosY += zoom*20*2
        env.PosX += zoom*20 
        cost[env.g[li+2,cj+1]]+=1
        changed=1
    # deplacement (2,-1)
    if touche == 'g' and li<env.nblignes-2 and cj > 0 and env.g[li+2,cj-1]>-1:
        env.PosY += zoom*20*2       
        env.PosX -= zoom*20                 
        cost[env.g[li+2,cj-1]]+=1
        changed=1
   # deplacement (1,2)
    if touche == 'j' and li<env.nblignes-1 and cj < env.nbcolonnes-2 and env.g[li+1,cj+2]>-1:
        env.PosY += zoom*20        
        env.PosX += zoom*20*2       
        cost[env.g[li+1,cj+2]]+=1
        changed=1
    # deplacement (1,-2)
    if touche == 'f' and li<env.nblignes-1 and cj >1 and env.g[li+1,cj-2]>-1:
        env.PosY += zoom*20
        env.PosX -= zoom*20*2           
        cost[env.g[li+1,cj-2]]+=1 
        changed=1


# La variable alea =1 si on veut des effets aleatoires sinon les transitions sont deterministes
    #On ajoute un effet aleatoire dans les transitions
    if env.alea==1 and changed==1:
        t=np.random.uniform(0,1)    
        if t>0.5:
            d=np.random.randint(8)
            dli=0
            if d== 0 or d==1 or d==2:
                dli=-1
            if d== 4 or d==5 or d==6:
                dli==1
            dcj=0
            if d==0 or d==7 or d==6:
                dcj=-1
            if d==2 or d==3 or d==4:
                dcj=1    
        # l'effet aleatoire est applique s'il cree un deplacement sur une case admissible     
            NewPosY = env.PosY+zoom*20*dli
            NewPosX = env.PosX+zoom*20*dcj        
            newcj=int((NewPosX-30)/(20*zoom))
            newli=int((NewPosY-30)/(20*zoom))   
            print('d',dli,dcj)
            if newli>=0 and newcj>=0 and newli<=env.nblignes-1 and newcj<=env.nbcolonnes-1 and env.g[newli,newcj]>-1:
                env.PosY=NewPosY
                env.PosX=NewPosX            
            
# on dessine le pion a sa nouvelle position
    Canevas.coords(Pion,env.PosX -9*zoom, env.PosY -9*zoom, env.PosX +9*zoom, env.PosY +9*zoom)       
    # env.globalcost=0    
    for k in range(6):
        env.globalcost+=cost[k]*env.weight[k]
    w.config(text='Cost = '+ str(env.globalcost))

# Structure 

In [4]:
class state:
    
    

    def __init__(self, coords, Value, reward, dictNStates, dictAroundStates, isReward):
        
        self.coords = coords
        self.value = Value
        self.reward = reward
        self.bestAction = ' '
        self.IsaReward = isReward
        self.dictNextStates = dictNStates
        self.nbNextStates = len(self.dictNextStates)
        self.dictAroundStates = dictAroundStates
        self.nbStatesAround = len(dictAroundStates)
        self.Probability = 1 - self.nbStatesAround/16 

    
        
    def print_state(self):

        print("coord: ",self.coords)
        print("value: ",self.value)
        print("reward: ",self.reward)
        print("dictNStates: ",self.dictNextStates)
        print("dictAroundStates: ",self.dictAroundStates)
        print("nbStatesAround: ",self.nbStatesAround)

In [5]:
class env:
    def __init__(self, nbcolonnes, nblignes, weight, alea, PmurDict, seed):
        self.nbcolonnes = nbcolonnes
        self.nblignes = nblignes
        self.g = np.zeros((nblignes,nbcolonnes),dtype=np.int)
        self.weight = weight

        
        # self.zoom = zoom
        self.alea = alea
        self.globalcost = 0
        self.PmurDict = PmurDict
        self.seed = seed
        
        self.PosX =20
        self.PosY =20
        self.genere_map_alea()
        self.listAllStates = self.create_all_states()

    def genere_map_alea(self):
        np.random.seed(self.seed)
        
        
        for i in range(self.nblignes):
            for j in range(self.nbcolonnes):
                z=np.random.uniform(0,1)
                if z < self.PmurDict['pmur']:
                    c=-1
                else:
                    if z < self.PmurDict['pmur']+ self.PmurDict['pblanc']:
                        c=0
                    else:    
                        if z < self.PmurDict['pmur']+ self.PmurDict['pblanc'] + self.PmurDict['pverte']:
                            c=1
                        else:
                            if z < self.PmurDict['pmur']+ self.PmurDict['pblanc'] +self.PmurDict['pverte'] + self.PmurDict['pbleue']:
                                c=2
                            else:
                                if z< self.PmurDict['pmur'] + self.PmurDict['pblanc'] + self.PmurDict['pverte'] + self.PmurDict['pbleue'] +self.PmurDict['prouge']:
                                    c=3
                                else:
                                    c=4   
                self.g[i,j]=c
        self.g[0,0]=0
        self.g[0,1]=0
        self.g[2,0]=0    
        self.g[self.nblignes-1,self.nbcolonnes-1]=0
        self.g[self.nblignes-2,self.nbcolonnes-1]=0
        self.g[self.nblignes-1,self.nbcolonnes-2]=0
        self.iBut = np.random.randint(2, self.nblignes-1)
        self.jBut = np.random.randint(2, self.nbcolonnes-1)
        self.g[self.iBut,self.jBut] = 5        

    def create_all_states(self):
        l = list()
        for i in range(self.nblignes):
            for j in range(self.nbcolonnes):
                coord =  (i,j)
                isaReward = False
                if self.g[i,j] < 0:
                    reward  = " mur "
                else:
                    #reward = np.format_float_scientific(self.weight[env.g[i,j]],exp_digits=1)
                    reward = -self.weight[self.g[i,j]]
                    if reward > 0: 
                        isaReward = True
                value = 0
                dictNStates = self.get_dictNextStates(i, j)
                dictAroundStates = self.get_around_States_Proba(i, j)
                
                
                s = state(coord, value, reward, dictNStates, dictAroundStates,isaReward )
                l.append(s)
        return l
    def get_dictNextStates(self,i,j):
        #check OutOfBound
        di = dict()
        if i-2>=     0       and j+1 <self.nbcolonnes and self.g[i-2,j+1] != -1:
            di['y']=(i-2,j+1)
        if i-1>=     0       and j+2 <self.nbcolonnes and self.g[i-1,j+2] != -1:
            di['u']=(i-1,j+2)
        if i+1<self.nblignes and j+2 <self.nbcolonnes and self.g[i+1,j+2] != -1:
            di['j']=(i+1,j+2)
        if i+2<self.nblignes and j+1 <self.nbcolonnes and self.g[i+2,j+1] != -1:
            di['h']=(i+2,j+1)
        if i+2<self.nblignes and j-1 >=       0       and self.g[i+2,j-1] != -1:
            di['g']=(i+2,j-1)
        if i+1<self.nblignes and j-2 >=       0       and self.g[i+1,j-2] != -1:
            di['f']=(i+1,j-2)
        if i-1>=     0       and j-2 >=       0       and self.g[i-1,j-2] != -1:
            di['r']=(i-1,j-2)
        if i-2>=     0       and j-1 >=       0       and self.g[i-2,j-1] != -1:
            di['t']=(i-2,j-1)

        return di
    def get_around_States_Proba(self, i, j):
        # check OutOfBound
        diStateAround = dict()
        if i+1<self.nblignes and j   <self.nbcolonnes and self.g[i+1,j  ] != -1:
            diStateAround['md'] = (i+1,j  , 1/16)

        if i-1>=      0      and j   <self.nbcolonnes and self.g[i-1,j  ] != -1:
            diStateAround['mg'] = (i-1,j  , 1/16)

        if i+1<self.nblignes and j+1 <self.nbcolonnes and self.g[i+1,j+1] != -1:
            diStateAround['hd'] = (i+1,j+1, 1/16)

        if i-1>=      0      and j-1 >=      0        and self.g[i-1,j-1] != -1:
            diStateAround['bg'] = (i-1,j-1, 1/16)

        if i+1<self.nblignes and j-1 >=      0        and self.g[i+1,j-1] != -1:
            diStateAround['bd'] = (i+1,j-1, 1/16)

        if i-1>=      0      and j+1 <self.nbcolonnes and self.g[i-1,j+1] != -1:
            diStateAround['hg'] = (i-1,j+1, 1/16)

        if i <self.nblignes  and j+1 <self.nbcolonnes and self.g[i  ,j+1] != -1:
            diStateAround['hm'] = (i  ,j+1, 1/16)

        if i <self.nblignes  and j-1 >=      0        and self.g[i  ,j-1] != -1:
            diStateAround['bm'] = (i  ,j-1, 1/16)
            
        return diStateAround

    def get_state(self,i,j):
        i = int(i)
        j = int(j)
        if i>=0 and i<self.nblignes and j >=0 and j<self.nbcolonnes:
            return self.listAllStates[i*self.nbcolonnes+j]
        return None

    def afficheV(self):
        print("Valeur de l'etat selon l'algo")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                value="{:.2e}".format(self.get_state(i,j).value)
                l.append(value)
            print(l)
          

    def afficheBestAction(self):
        print("meilleure Action")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).bestAction)
            print(l)

    def afficheP(self):
        print("Proba d'arriver ce State")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).Probability)
            print(l)

    def afficheNBSA(self):
        print("NB State Around")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).nbStatesAround)
            print(l)

    def afficheR(self):
        print("Rewards")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append(self.get_state(i,j).reward)
            print(l)
    def afficheNBNS(self):
        print("NB Next State ")
        for i in range(self.nblignes):
            l = list()
            for j in range(self.nbcolonnes):
                l.append((self.get_state(i,j).nbNextStates,self.get_state(i,j).dictNextStates))
            print(l)



In [13]:
zoom = 2

#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 0 
#taille de la grille
nblignes = 10
nbcolonnes = 20 



# valeurs de la grille
weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 10
weight[2] = 20
weight[3] = 30
weight[4] = 40
weight[5] = -1000

seed = 0
PmurDict=dict()
PmurDict['pmur'] = 0.15 
PmurDict['pblanc'] = 0.45 
PmurDict['pverte'] = 0.1
PmurDict['pbleue'] = 0.1
PmurDict['prouge'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante

e = env(nbcolonnes,nblignes,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed)
e.genere_map_alea()
print(e.g)
# e.afficheV()
# e.afficheP()
# e.afficheNBSA()
# e.afficheR()
# e.afficheBestAction()
# e.afficheNBNS() 
Mafenetre = create_Visualization('test',e, zoom)
Mafenetre.mainloop()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_9730/3507122743.py:12: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  weight = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_9730/3994344422.py:5: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current 

[[ 0  0  1  0  0  1  0  3  4  0  2  0  0  4 -1 -1 -1  3  2  3]
 [ 4  2  0  2 -1  1 -1  4  0  0  0  2  0  0 -1  1  1  1  4  1]
 [ 0  0  1 -1  1  1  0 -1  0  0  0  0  4 -1  0  0  1  0  0  0]
 [ 0 -1  1 -1  0  0  3 -1  3 -1  4  0  4  1  2 -1  0 -1  0 -1]
 [ 0  0 -1  1  0  0  0 -1  0  4  0  1 -1  2  0  0  0 -1  3 -1]
 [ 1  0  2  4  0  0  0  0  0  4  0  3  1  0  3  0  3  0  3  1]
 [ 2  0  4  1  0  1 -1  0  1  0  1  0 -1  0  0  0  0  1  1  0]
 [ 3  0  0  3  3  2 -1  4  2  4 -1  3  0  1 -1  3  3  0  0 -1]
 [ 1  0  2  3  4  3 -1  0  2  5  0 -1  0 -1  2  0  0  4  2  0]
 [ 0  1  0  0  4  1  0  0  2  0  0  0  0  4  2  0  0  0  0  0]]


# Partie 1 Recherche d'un trajectoire prudente

## question A
 Modéliser le problème comme un processus décisionnel Markovien en donnant une récompense significative lorsqu’on atteint la case but, par exemple un bonus (réduction de coût) de 1000 points mais on pourra essayer d’autres valeurs en fonction de la dimension de la grille choisie. Ecrire les équations de Bellman définissant la valeur  vij d’être dans une case (i, j) pour une politique stationnaire optimale et un facteur d’actualisation γ.

## question B
Dans un premier temps on se place dans le cas d éterministe (la case cible est atteinte avec une probabilité 1 à chaque saut). Déterminer par itération de la valeur un chemin optimal de la case initiale vers la case but et tester sur une grille de taille puis 5×10, 10×15, 15×20 (on essaiera avec γ = 1 et γ = 0.5). On donnera les grilles en visualisant les trajectoires optimales et on donnera les temps de calcul.

### Code

In [7]:
def value_iterationD(nbcolonnes,nblignes, gamma, epsilon):
    Value_states = np.zeros(nblignes*nbcolonnes, dtype=np.int)
    e = env(nbcolonnes,nblignes,g,weight=weight)
    
    e.afficheV()
    e.afficheP()
    e.afficheP()
    # actions = ['f','g','h','j','y','u',]
    iteration = 0
    stop = False
    while stop==False:
        
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward == True:
                
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                somme = 0
                pState = e.get_state(v[0],v[1]) 
                somme = pState.Probability*pState.value 
                #print(pState.Probability)             
                for cState in pState.dictAroundStates.values():
                    
                    sA = e.get_state(cState[0],cState[1])
                    somme+= (1/16*sA.value)
                        
                temp = s.reward+gamma*somme

                if temp > best:
                    best = temp
                    saveA = a
                
                
            s.bestAction = saveA
            s.value = best
            # print(tempBest)
            # for i in tempBest :
            #     bestV = 0
            #     bestA = 'no'
            #     if i[1]>bestV :
            #         bestV = i[1]
            #         bestA = i[0]

            # s.value = bestV
            # s.bestAction = bestA
            
          
            
            
        iteration +=1
       

        if iteration>1000:
            stop = True
            e.afficheV()
            e.afficheBestAction()
            e.afficheR()
            print(iteration)
    

### TEST

## question C
On se place maintenant dans le cas de transitions aléatoires définies comme expliqué ci-dessus. On souhaite déterminer une politique optimale par itération de la valeur et effectuer des essais numériques de résolution de grilles de différentes tailles. Pour chaque taille on tirera 10 instances de grilles aléatoirement et on donnera dans un tableau le temps moyen de résolution et le nombre moyen d’itérations. On pourra également étudier l’impact de γ en faisant varier γ = 0.9, γ = 0.7, γ = 0.5.

### Code

In [11]:
def value_iterationG(env, gamma, epsilon):
    Value_states = np.zeros(env.nblignes*env.nbcolonnes, dtype=np.int)
    e = env
    
    e.afficheV()
    e.afficheP()
    e.afficheP()
    # actions = ['f','g','h','j','y','u',]
    iteration = 0
    stop = False
    while stop==False:
        
        for count, s in enumerate(e.listAllStates):
            best = float('-inf')
            saveA = ' '
            
            if s.reward == " mur ":
                continue
            elif s.IsaReward == True:
                
                s.bestAction = 'R'
                continue
            for a, v in s.dictNextStates.items():
                somme = 0
                pState = e.get_state(v[0],v[1]) 
                somme = pState.Probability*pState.value 
                #print(pState.Probability)             
                for cState in pState.dictAroundStates.values():
                    
                    sA = e.get_state(cState[0],cState[1])
                    somme+= (1/16*sA.value)
                        
                temp = s.reward+gamma*somme

                if temp > best:
                    best = temp
                    saveA = a
                
                
            s.bestAction = saveA
            s.value = best
            # print(tempBest)
            # for i in tempBest :
            #     bestV = 0
            #     bestA = 'no'
            #     if i[1]>bestV :
            #         bestV = i[1]
            #         bestA = i[0]

            # s.value = bestV
            # s.bestAction = bestA
            
          
            
            
        iteration +=1
       

        if iteration>1000:
            stop = True
            e.afficheV()
            e.afficheBestAction()
            e.afficheR()
            print(iteration)
    

gamma = 0.9
epsilon = 1




# Mafenetre.mainloop()
# Mafenetre.destroy()

### TEST

In [12]:

#transitions aleatoires si alea =1 sinon mettre alea=0
alea = 0 
#taille de la grille
nblignes = 10
nbcolonnes = 20 

cost = np.zeros(6,dtype=np.int)

# valeurs de la grille
weight = np.zeros(6,dtype=np.int)
weight[0] = 1
weight[1] = 10
weight[2] = 20
weight[3] = 30
weight[4] = 40
weight[5] = -1000

seed = 0
PmurDict=dict()
PmurDict['pmur'] = 0.15 
PmurDict['pblanc'] = 0.45 
PmurDict['pverte'] = 0.1
PmurDict['pbleue'] = 0.1
PmurDict['prouge'] = 0.1
#  pnoire=0.1 mais pas besoin de le specifier c'est la couleur restante

e = env(nbcolonnes,nblignes,weight=weight,alea=alea,PmurDict=PmurDict, seed=seed)
e.genere_map_alea()
print(e.g)
gamma = 0.9
epsilon = 1

value_iterationG(env=e, gamma= gamma, epsilon=epsilon)


Mafenetre.mainloop()
# Mafenetre.destroy()

/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_9730/2563312665.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cost = np.zeros(6,dtype=np.int)
/var/folders/00/wvw3l7pn6tg9wsdl40t3xxn40000gn/T/ipykernel_9730/2563312665.py:10: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current us

[[ 0  0  1  0  0  1  0  3  4  0  2  0  0  4 -1 -1 -1  3  2  3]
 [ 4  2  0  2 -1  1 -1  4  0  0  0  2  0  0 -1  1  1  1  4  1]
 [ 0  0  1 -1  1  1  0 -1  0  0  0  0  4 -1  0  0  1  0  0  0]
 [ 0 -1  1 -1  0  0  3 -1  3 -1  4  0  4  1  2 -1  0 -1  0 -1]
 [ 0  0 -1  1  0  0  0 -1  0  4  0  1 -1  2  0  0  0 -1  3 -1]
 [ 1  0  2  4  0  0  0  0  0  4  0  3  1  0  3  0  3  0  3  1]
 [ 2  0  4  1  0  1 -1  0  1  0  1  0 -1  0  0  0  0  1  1  0]
 [ 3  0  0  3  3  2 -1  4  2  4 -1  3  0  1 -1  3  3  0  0 -1]
 [ 1  0  2  3  4  3 -1  0  2  5  0 -1  0 -1  2  0  0  4  2  0]
 [ 0  1  0  0  4  1  0  0  2  0  0  0  0  4  2  0  0  0  0  0]]
Valeur de l'etat selon l'algo
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00']
['0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '0.00e+00', '

### R

## Question D 
Pour une grille donné on essayera de piloter à la main quelques trajectoires menant à la case but et on comparera leur coût au coût moyen d’une politique mixte optimale observé sur 20 executions successives de cette politique sur la même grille. On pourra aussi comparer les coûts moyens observés avec la valeur espérée de la politique mixte optimale testée.

# Partie 2 Recherche d'une politique équilibré 

## A
Dans cette partie on considère des grilles contenant environ 15% de cases interdites, ainsi que 20% de
cases rouges et 20% de cases bleues, les couleurs vertes et noires n’étant plus présentes.
a) On cherche une politique optimale équilibrée entre cases rouges et cases bleues pour atteindre l’état but
à partir de l’état initial. Pour cela on utilise des coûts immédiats à deux composantes (c1, c2), de telle sorte
que faire un saut depuis une case blanche engendre un coût immédiat de (1, 1), faire un saut depuis une case
bleue engendre un coût immédiat (2, 0) et faire un saut depuis une case rouge engendre un coût immédiat de
(0, 2). Par exemple, une trajectoire qui fait 22 sauts de cavaliers dont 10 depuis une case blanche, 8 depuis
une case bleue et 4 depuis une case rouge aura pour vecteur coût (10 + 6 × 2, 10 + 4 × 2) = (22, 18), auquel
il faut retirer le bonus final d’atteindre la case but sur chacune des deux composantes. Une trajectoire
traversant autant de cases bleues que de cases rouges aura un vecteur de coûts cumulés équilibré. Pour
évaluer une politique π, on utilise le vecteur (c1(π), c2(π)) qui représentent les coûts espérés actualisés de
la politique π. On souhaite trouver une politique qui optimise le critère suivant :

                                                            min max{c1(π), c2(π)}
                                                             π
                     
Expliquer pourquoi on ne peut utiliser l’itération de la valeur pour la résolution de ce problème. Proposer
alors une approche de résolution reposant sur la formulation d’un MDP bi-objectif et sa résolution par
programmation mathématique pour déterminer une politique mixte optimale.

## B
Tester la résolution pratique du problème de recherche d’une trajectoire équilibrée entre le bleu et le
rouge sur des grilles de différentes tailles avec l’approche proposée à la question précédente. On donnera
ici encore les temps moyens de résolution. Simuler les politiques optimales obtenues en les éxécutant 20
fois sur la même instance. Calculer le vecteur moyen obtenu (c1, c2) qui représente l’espérance de coût
empiriquement observée de la politique optimale selon les deux critères considérés.

## C
Comment modifier l’approche proposée en a) pour déterminer une politique pure optimale au sens du
critère (1) ? Tester cette approche et comparer ses performances à celles trouvées en b) sur les mêmes
instances de problème

# Test

In [10]:


Mafenetre.mainloop()


NameError: name 'initialize' is not defined